### KMP

In [15]:
# s1, s2字符串匹配
def KMP(s1, s2):
    if not s1 or not s2 or len(s2) > len(s1):
        return -1
    str1 = list(s1)
    str2 = list(s2)
    nexts = getNextsArray(str2)
#     print(nexts)
    x = 0
    y = 0
    while x < len(str1) and y < len(str2):
        if str1[x] == str2[y]:
            x += 1
            y += 1
        elif nexts[y] >= 0:  
            y = nexts[y]
        else:   #此时y=0，nexts[y] = -1, 即x与y的第一个数也没碰上
            x += 1
    return x - y if y == len(str2) else -1
            
    
def getNextsArray(arr):
    if len(arr) == 1:
        return [-1]
    nexts = [None] * len(arr)
    nexts[0] = -1
    nexts[1] = 0
    index = 2
    pointer = 0
    while index < len(arr):
        if arr[index - 1] == arr[pointer]:
            pointer += 1
            nexts[index] = pointer
            index += 1
        elif pointer > 0:
            pointer = nexts[pointer]
        else:
            nexts[index] = 0
            index += 1
    return nexts
            

In [16]:
#利用i位置之前的位置信息加速
def getNextArray(s):
    if len(s) == 1:
        return [-1]
    nexts = [None] * len(s)
    nexts[0] = -1
    nexts[1] = 0
    index = 2 #当前填到哪，利用index-1跳步
    cn = 0   # 当前是哪个位置，和index-1位置字符比对
    while index < len(s):
        if s[index - 1] == s[cn]:
            nexts[index] = cn + 1
            cn += 1 #index的cn，给index+1使用
            index += 1
        elif cn > 0: #继续跳 #cn没对上，但查看cn-1当时对到了哪个cn
            cn = nexts[cn]
        else:
            nexts[index] = 0 #此时cn已经跳到了0，不用再动了
            index += 1
    return nexts
    
def getIndexOf(s1, s2):
    if not s1 or not s2 or len(s1) < len(s2):
        return -1
    str1 = list(s1)
    str2 = list(s2)
    #比对到的位置
    x = 0
    y = 0
    nexts = getNextArray(str2)
    #O(N)
    #证明：(做差评估浮动幅度)
    #x的最大值是N，y的最大值也是N。X-Y的最大值也是N
    #分支一if时， x++，y++ ==> x变大， x-y不变。
    #分支二elif时，x++     ==> x变大， x-y变大。
    #分支三else时，y变小，  ==> x不变， x-y变大。
    #所以三个分支的浮动，<=2N
    while x < len(str1) and y < len(str2):
        if str1[x] == str2[y]:
            x += 1
            y += 1
        elif nexts[y] == -1: # y=0
            x += 1
        else:
            y = nexts[y]
    return x - y if y == len(str2) else -1 #只有x=y,y++,所以y来到越界，一定y遍历了整个s2.
   


In [97]:
import random
def generateRandomString(maxLen, maxValue=25):
    string = ""
    for i in range(maxLen):
        string += (chr(random.randint(0, maxValue) + ord("a")))
    return string

testTime = 10000
maxValue = 3
maxLen = 6
succeed=True
for _ in range(testTime):
    value = random.randint(1, maxValue)
    length = random.randint(0, maxLen)
    s1 = generateRandomString(length * 10, value)
    s2 = generateRandomString(length - random.randint(0, length), value - random.randint(0, value))
    ans1 = getIndexOf(s1,s2)
    ans2 = KMP(s1,s2)
    if ans1 != ans2:
        print(ans1, ans2)
        print("something wrong.")
        succeed = False
        break
if succeed:
    print("succeed.")

succeed.


### Manacher
https://leetcode.cn/problems/longest-palindromic-subsequence/

In [133]:
def manacher(string):
    if not string:
        return 0
    s = manacherString(string)
    R = -1
    C = -1
    index = 0
    N = len(s)
    pArr = [None] * N
    while index < N:
        if index < R:
            pArr[index] = min(R - index, pArr[2*C-index])
        else:
            pArr[index] = 1
        while index - pArr[index] >= 0 and index + pArr[index] < N and s[index - pArr[index]] == s[index + pArr[index]]:
            pArr[index] += 1
        if index + pArr[index] > R:
            R = index + pArr[index]
            C = index
        index += 1
    p = 0
    start = 0
    for i in range(N):
        if pArr[i] > p:
            p = pArr[i]
            if i % 2 == 0:
                start = i // 2 - (p - 1)//2
            else:
                start = (i // 2 - (p - 2) // 2)
    return string[start:start+p-1] #返回最长子串
         
    
def manacherString(string):
    stringCopy = ""
    for i in string:
        stringCopy += str(i)
    s = ["#"]
    for i in stringCopy:
        s.append(i)
        s.append("#")
    return s

In [139]:
manacher("aabcb")

['#', 'a', '#', 'a', '#', 'b', '#', 'c', '#', 'b', '#']
[1, 2, 3, 2, 1, 2, 1, 4, 1, 2, 1]


'bcb'

### 无序数组中求第K小的数

#### 方法一：改写快排

In [167]:
import random
def topK(arr, k):
    if not arr or k > len(arr):
        return None
    N = len(arr)
    r = random.randint(0, N-1)
    swap(arr, r, N-1)
    equalL, equalR = partition(arr, 0, N-1)
    while not (equalL <= k - 1 <= equalR):
        if k < equalL:
            L = 0
            R = equalL - 1
        elif k > equalR:
            L = equalR
            R = N - 1
        r = random.randint(L, R)
        swap(arr, r, R)
        equalL, equalR = partition(arr, L, R)
    return arr[equalL]
def partition(arr, L, R):
    p1 = L - 1
    p2 = R
    index = L
    while index < p2:
        if arr[index] < arr[R]:
            p1 += 1
            swap(arr, index, p1)
            index += 1
        elif arr[index] == arr[R]:
            index += 1
        else:
            p2 -= 1
            swap(arr, index, p2)
    swap(arr, index, R)
    return p1+1, p2

def swap(arr, i, j):
    temp = arr[i]
    arr[i] = arr[j]
    arr[j] = temp
    return

In [169]:
def bfprt(arr, L, R, index):
    pass

#### 无序数组topK个最大的数¶

### Morris遍历

#### 先序遍历

In [291]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def morrisPre(head):
    if not head:
        return None
    cur = head
    while cur:
        mostRight = cur.left
        if mostRight:
            while mostRight.right and mostRight.right != cur:
                mostRight = mostRight.right
            if not mostRight.right:
                mostRight.right = cur #once
                print(cur.val, end=",")
                cur = cur.left
                continue
            else:
                mostRight.right = None # twice
        else:
            print(cur.val, end=",")
        cur = cur.right
    return None

#### 中序遍历

In [292]:
def morrisIn(head):
    if not head:
        return None
    cur = head
    while cur:
        mostRight = cur.left
        if mostRight:
            while mostRight.right and mostRight.right != cur:
                mostRight = mostRight.right
            if not mostRight.right:
                mostRight.right = cur #once
                cur = cur.left
                continue
            else:
                mostRight.right = None # twice
        
        print(cur.val, end=",")
        cur = cur.right
    return None

#### 后序遍历

In [245]:
def morrisPosPrac(head):
    if not head:
        return None
    cur = head
    while cur:
        left = cur.left
        mostRight = cur.left
        if mostRight:
            while mostRight.right and mostRight.right != cur:
                mostRight = mostRight.right
            if not mostRight.right:
#                 print("connect", cur.val)
                mostRight.right = cur #once
                cur = cur.left
                continue
            else:
                mostRight.right = None # twice
#                 print("now",cur.val)
                reversePrac(left)
                printEdgePrac(mostRight)
                reversePrac(mostRight)
                cur.left = left
        cur = cur.right
    end = reverse(head)
    printEdgePrac(end)
    head = reverse(end)
    return None

def reversePrac(head):
    if not head:
        return None
    pre = None
    nxt = None
    cur = head
    while cur:
        nxt = cur.right
        cur.right = pre
        pre = cur
        cur = nxt
    return pre

def printEdgePrac(head):
    if not head:
        return None
    cur = head
    while cur:
        print(cur.val, end=",")
        cur = cur.right
    return None



In [242]:
def morrisPos(head):
    if not head:
        return None
    cur = head
    mostRight = None
    while cur:
        mostRight = cur.left
        if mostRight:
            while mostRight.right and mostRight.right != cur:
                mostRight = mostRight.right
            if not mostRight.right:
                mostRight.right = cur
                cur = cur.left
                continue
            else:
                mostRight.right = None
                printEdge(cur.left)
        cur = cur.right
    printEdge(head)
    return None

def printEdge(head):
    tail = reverseEdge(head)
    cur = tail
    while cur:
        print(cur.val, end=",")
        cur = cur.right
    reverseEdge(tail)
    return None

def reverseEdge(head):
    pre = None
    nxt = None
    while head:
        nxt = head.right
        head.right = pre
        pre = head
        head = nxt
    return pre

In [404]:
import random
from collections import deque
def generateRandBT(depth, maxValue):
    if depth == 1:
        return Node(random.randint(-maxValue, maxValue))

    head = Node(random.randint(-maxValue, maxValue))
    cur_last = head
    next_last = None
    queue = deque()
    queue.append(head)
    while queue and depth > 0:
        cur = queue.popleft()
        rand = random.random()
        if rand > 0.3:
            cur.left = Node(random.randint(-maxValue, maxValue))
            queue.append(cur.left)
            next_last = cur.left
        rand = random.random()
        if rand > 0.3:
            cur.right = Node(random.randint(-maxValue, maxValue))
            queue.append(cur.right)
            next_last = cur.right
        if cur == cur_last:
            depth -= 1
            cur_last = next_last
    return head

In [209]:
def getSpace(num):
    space = ""
    for i in range(num):
        space+= " "
    return space
    

def printInOrder(head, height, to, length):
    if head is None:
        return 
    printInOrder(head.right, height+1, "↓", length)
    val = to + str(head.val) + to
    lenM= len(str(head.val))
    lenL = (length - lenM) // 2
    lenR = length - lenM - lenL
    val = getSpace(lenL) + val + getSpace(lenR)
    print(getSpace(height * length) + val)
    printInOrder(head.left, height+1, "↑", length)
def printBinaryTree(head):
    print("Binary Tree: ")
    printInOrder(head, 0, "H", 17)
    print()

In [371]:
t = generateRandBT(3, 10)

In [372]:
printBinaryTree(t)

Binary Tree: 
                                                           ↓2↓        
                                         ↓-7↓        
                                                           ↑0↑        
                        ↓-2↓        
                                                           ↓0↓        
                                         ↑-6↑        
                                                          ↑-4↑        
        H0H        
                                                          ↓-7↓        
                                         ↓-10↓       
                                                          ↑-3↑        
                        ↑-8↑        
                                                          ↓-1↓        
                                          ↑0↑        
                                                           ↑5↑        



In [246]:
morrisPos(t)

-9,-9,1,0,-4,4,7,2,

In [247]:
morrisPosPrac(t)

-9,-9,1,0,-4,4,7,2,

#### 判断一棵树是否是搜索二叉树

In [249]:
def morrisIn(head):
    if not head:
        return None
    cur = head
    preVal = None
    flag = True
    while cur:
        mostRight = cur.left
        if mostRight:
            while mostRight.right or mostRight.right != cur:
                mostRight = mostRight.right
            if not mostRight.right:
                mostRight.right = cur #once
                cur = cur.left
                continue
            else:
                mostRight.right = None # twice
        
        if pre is not None and pre.val > cur.val:
            flag = False
        preVal = cur.val
        cur = cur.right
    return flag

#### 求以head为头的二叉树树中，最小深度是多少

In [254]:
import sys
def minHeight(head):
    if not head:
        return 0
    cur = head
    curLevel = 0
    minDepth = sys.maxsize
    while cur:
        mostRight = cur.left
        if mostRight:
            leftCount = 1
            while mostRight.right and mostRight.right != cur:
                leftCount += 1
                mostRight = mostRight.right
            if not mostRight.right:
                curLevel += 1
                mostRight.right = cur
                cur = cur.left
                continue
            else:
                if not mostRight.left: #mostRight.right == cur:
                    minDepth = min(minDepth, curLevel)#此时curLevel仍然是左树右边界的深度
                mostRight.right = None
                curLevel -= leftCount
                cur = cur.right  
        else:
            curLevel += 1
            cur = cur.right
    cur = head
    depth = 1
    while cur.right:
        cur = cur.right
        depth += 1
    if not cur.left:
        minDepth = min(minDepth, depth)
    return minDepth
    

In [255]:
minHeight(t)

4

In [396]:
'''
思路：
问题1. 要搞定当前节点在第几层
问题2. 判断是否是叶节点 + 每次尝试更新最低的层数

解决问题1:
情况1. X的层数为7，当Y为X的左孩子，则Y的层数一定为7+1=8；
情况2.           当Y为X的右孩子，不一定！因为Y有可能是X向上指回去时的右孩子(要第二次回到Y的时候)：
情况2.1          当Y的左树的最右节点不是X时，层数为7+1=8
情况2.2          当Y的左树的最右节点是X时，X的左树所在右边界有几个节点(Z)，7-Z即为层数。

解决问题2:因为改过节点，无法通过无双子判断是否叶节点
对于能回到两次的节点，在第二次恢复左树最右节点的右节点后，再看其是否是叶节点。
最后单独检查整棵树的最右边界是否是叶节点

'''
import sys
def minDepth(head):
    if not head:
        return 0
    cur = head
    mostRight = None
    curLevel = 0
    minHeight = sys.maxsize
    while cur:
        mostRight = cur.left
        if mostRight:
            rightBoardSize = 1
            while mostRight.right and mostRight.right != cur:
                rightBoardSize += 1
                mostRight = mostRight.right
            if not mostRight.right:
                curLevel += 1
                mostRight.right = cur
                cur = cur.left
                continue
            else:
                if not mostRight.left:
                    minHeight = min(minHeight, curLevel)
                curLevel -= rightBoardSize
                mostRight.right = None
        else:
            curLevel += 1
        cur = cur.right
    finalRight = 1
    cur = head
    while cur.right:
        finalRight += 1
        cur = cur.right
    if not cur.left and not cur.right:
        minHeight = min(minHeight, finalRight)
    return minHeight

In [405]:
testTime = 10000
maxValue = 100
maxDepth = 20
succeed = True
for _ in range(testTime):
    value = random.randint(1, maxValue)
    depth = random.randint(1, maxDepth)
#     print(depth)
    tree = generateRandBT(depth, value)
    ans1 = minHeight(tree)
    ans2 = minDepth(tree)  
    if ans1 != ans2:
        print(ans1, ans2)
        printBinaryTree(tree)
        succeed = False
        break
if succeed:
    print("success.")

success.


In [406]:
printBinaryTree(tree)

Binary Tree: 
                        ↓-15↓       
                                          ↑8↑        
                                                           ↑5↑        
        H7H        



In [409]:
morrisPos(t)

5,-1,0,-3,-7,-10,-8,-4,0,-6,0,2,-7,-2,0,

In [411]:
morrisIn(t)

5,0,-1,-8,-3,-10,-7,0,-4,-6,0,-2,0,-7,2,